In [1]:
import tensorflow as tf
tf.python.control_flow_ops = tf

import os
import pandas as pd
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, \
MaxPooling2D, Conv2D, Lambda, Cropping2D, Convolution2D,\
AveragePooling2D
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint
from sklearn.cross_validation import train_test_split

from copy import deepcopy
from sklearn.utils import shuffle


from generator import generator
from get_images import get_images

from keras.optimizers import Adam


Using TensorFlow backend.
/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
datadirs=['/notebooks/udacity/new_training/map1_backward/',
                 '/notebooks/udacity/new_training/map1_forward/',
                 '/notebooks/udacity/new_training/map1_recovery_backward/',
                 '/notebooks/udacity/new_training/map1_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_forward/',
                 '/notebooks/udacity/new_training/map2_backward/',
                 '/notebooks/udacity/new_training/map2_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_recovery_backward/',
                   '/notebooks/udacity/new_training/map1_error_correction/',
                   '/notebooks/udacity/new_training/map2_error_correction/'
         ]

#datadirs=['/notebooks/udacity/car_data/data/']

images=get_images(datadirs,0.1)

In [3]:
#different preprocessing methods
size=(40,80)

def preprocessing_resize(im):
    return cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2YUV),(size[1],size[0]))

def preprocessing_yuv(im):
    return cv2.cvtColor(im, cv2.COLOR_BGR2YUV)



In [4]:
from car_models import *

In [5]:
from generator import generator

In [6]:
names, steering, inverse = images.img.values, images.real.values, images.inverse.values

In [7]:
from sklearn.cross_validation import train_test_split 

names_train, names_valid, steering_train, steering_valid, inverse_train, inverse_valid = \
    train_test_split(names, steering, inverse, test_size=0.2)
    
names_valid, names_test, steering_valid, steering_test, inverse_valid, inverse_test=\
train_test_split(names_valid, steering_valid, inverse_valid, test_size=0.5)

In [12]:
train_gen=generator(names_train, steering_train, batch_size=128)
valid_gen=generator(names_valid, steering_valid, batch_size=128)
test_gen=generator (names_test, steering_test, batch_size=128)
    

In [9]:
grid=[{'get_model':modified_lenet, 'correction':0.09, \
       'preprocessing':lambda x:x,'optimizer':'adam','gpu':'0','name':'lenet_adam'},\
      {'get_model':modified_vgg, 'correction':0.09, \
       'preprocessing':lambda x:x,'optimizer':'adam','gpu':'0','name':'mod_adam_adam'},\
     {'get_model':nvidia_net, 'correction':0.09, \
       'preprocessing':preprocessing_yuv,'optimizer':'adam','gpu':'0','name':'nvidia_net_02_adam'},
      {'get_model':inception, 'correction':0.09, \
       'preprocessing':lambda x:x,'optimizer':'adam','gpu':'0','name':'inception_02_adam'},
     {'get_model':vgg16, 'correction':0.09, \
       'preprocessing':lambda x:x,'optimizer':'adam','gpu':'0','name':'vgg16_02_adam'},
      ]

In [29]:
import os
import pickle

def train_model(get_model, correction, preprocessing, name, optimizer, gpu):
    import os
    print ('training', name)
    os.environ['CUDA_VISIBLE_DEVICES']=str(gpu)
    
    model=get_model()
    model.compile(loss='mse', optimizer=optimizer, metrics=['mean_squared_error'])
    
    if not os.path.exists('results/logs_%s'%name):
        os.mkdir('results/logs_%s'%name)
    
    
    checkpoint = ModelCheckpoint("results/model_%s.h5" % name, monitor='val_mean_squared_error', verbose=1,
                              save_best_only=True, mode='min')
    early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                               min_delta=0.001, patience=3,
                                verbose=1, mode='min')

    history = History()

    tb=TensorBoard(log_dir='results/logs_%s'%name, histogram_freq=0, write_graph=True, write_images=False)

    
    ##todo - nb_epoch=100
    q=model.fit_generator(train_gen, \
                    samples_per_epoch=len(names_train),\
                    nb_epoch=100,\
                    verbose=1,\
                    validation_data=valid_gen, \
                    nb_val_samples=len(names_valid),\
                    callbacks=[checkpoint, early_stop,history,tb]\
                   )
    
    f=open('results/history_%s.pk1' % name,'wb')
    pickle.dump(history.history,f,-1)
    f.close()
    
    model.load_weights("results/model_%s.h5" % name)
    
    res=model.evaluate_generator(test_gen,val_samples=len(names_test))[0]
    return q,res

In [ ]:
res=[train_model(**a) for a in grid[1:]]

training mod_adam_adam
Instructions for updating:
Please switch to tf.summary.merge_all.
Epoch 1/100
220416/220492 [============================>.] - ETA: 0s - loss: 72.2662 - mean_squared_error: 72.2662         

/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_mean_squared_error improved from inf to 0.07141, saving model to results/model_mod_adam_adam.h5
220544/220492 [==============================] - 1269s - loss: 72.2243 - mean_squared_error: 72.2243 - val_loss: 0.0714 - val_mean_squared_error: 0.0714
Epoch 2/100
220544/220492 [==============================] - 1273s - loss: 0.0795 - mean_squared_error: 0.0795 - val_loss: 0.0644 - val_mean_squared_error: 0.0644
Epoch 3/100
220544/220492 [==============================] - 1272s - loss: 0.0762 - mean_squared_error: 0.0762 - val_loss: 0.0569 - val_mean_squared_error: 0.0569
Epoch 4/100
 63488/220492 [=======>......................] - ETA: 894s - loss: 0.0775 - mean_squared_error: 0.0775 

In [28]:
res

[(0, 0.026067847110055112)]

In [18]:
grid[:1]

[{'correction': 0.09,
  'get_model': <function car_models.modified_lenet>,
  'gpu': '0',
  'name': 'lenet_adam',
  'optimizer': 'adam',
  'preprocessing': <function __main__.<lambda>>}]